In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
olmo = AutoModelForCausalLM.from_pretrained("allenai/OLMo-1B-hf")
tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-1B-hf")

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
olmo.eval()

OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoDecoderLayer(
        (self_attn): OlmoSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): OlmoRotaryEmbedding()
        )
        (mlp): OlmoMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): OlmoLayerNorm()
        (post_attention_layernorm): OlmoLayerNorm()
      )
    )
    (norm): OlmoLayerNorm()
  )
  (

In [6]:
input_text1 = "Can brain cells move? By movement I mean long distance migration (preferably within the brain only)."
input_text2 = "Is it possible for brain cells to migrate over long distances, specifically within the brain itself?"

In [7]:
inputs1 = tokenizer(input_text1, return_tensors='pt')
inputs2 = tokenizer(input_text2, return_tensors='pt')

In [ ]:
inputs1 = {k: v.to(device).requires_grad_(True) for k, v in inputs1.items()}
inputs2 = {k: v.to(device).requires_grad_(True) for k, v in inputs2.items()}

In [ ]:
output1 = olmo(**inputs1)
output2 = olmo(**inputs2)

In [ ]:
# todo: investigate how olmo uses a single training iteration, check masking
# todo: add filtering with regard to open instruct (threshold for similarity)
# todo: ranking between sampling
# todo: tf-idf -> term-frequency inverse-document-frequency
# todo: think about explainability vs. similarity